In [34]:

# 2020-04-13 at Office
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.ndimage
from scipy.ndimage import gaussian_filter
import os, sys
import glob

In [35]:
path = '/media/joshua/HDD_fun2/1M_library_split/val_npy/polSa+0.94dump_00001152_160.372_109.531_10_6.375e+09_9.27725e+26_8.97031_1.75016.h5.npy'

pol_image = np.load(path)
print(pol_image.shape)

(160, 160, 5)


In [36]:
def npy_save_txt(img, fname, xdim = 160, ydim = 160, source = 'M87', mjd=False, time=False):
    """Save image data to text file.
       Args:
            fname (str): path to output text file
            mjd (int): MJD of saved image
            time (float): UTC time of saved image
       Returns:
    """

    # Transform to Stokes parameters:

    RA = '12 h 30 m 49.4234 s'

    DEC = '12 deg 23 m 28.0437 s'

    # MJD: 0.000000 

    RF: 227.0707 

    # FOVX: 180 pix 0.000180 as 

    # Coordinate values
    pdimas = 10**(-6)#psize/RADPERAS
    xs = np.array([[j for j in range(xdim)] for i in range(ydim)]).reshape(xdim*ydim,1)
    xs = pdimas * (xs[::-1] - xdim/2.0)
    ys = np.array([[i for j in range(xdim)] for i in range(ydim)]).reshape(xdim*ydim,1)
    ys = pdimas * (ys[::-1] - xdim/2.0)
    
    imvec = img[:, :, 0]
    qvec = img[:, :, 1]
    uvec = img[:, :, 2]
    vvec = img[:, :, 3]

    # If V values but no Q/U values, make Q/U zero
    if len(vvec) and not len(qvec):
        qvec = 0*vvec
        uvec = 0*vvec

    # Format Data
    if len(qvec) and len(vvec):
        outdata = np.hstack((xs, ys, (imvec).reshape(xdim*ydim, 1),
                                     (qvec).reshape(xdim*ydim, 1),
                                     (uvec).reshape(xdim*ydim, 1),
                                     (vvec).reshape(xdim*ydim, 1)))
        hf = "x (as)     y (as)       I (Jy/pixel)  Q (Jy/pixel)  U (Jy/pixel)  V (Jy/pixel)"

        fmts = "%10.10f %10.10f %10.10f %10.10f %10.10f %10.10f"

    elif len(qvec):
        outdata = np.hstack((xs, ys, (imvec).reshape(xdim*ydim, 1),
                                     (qvec).reshape(xdim*ydim, 1),
                                     (uvec).reshape(xdim*ydim, 1)))
        hf = "x (as)     y (as)       I (Jy/pixel)  Q (Jy/pixel)  U (Jy/pixel)"

        fmts = "%10.10f %10.10f %10.10f %10.10f %10.10f"

    else:
        outdata = np.hstack((xs, ys, (imvec).reshape(xdim*ydim, 1)))
        hf = "x (as)     y (as)       I (Jy/pixel)"
        fmts = "%10.10f %10.10f %10.10f"

    # Header
    if not mjd: mjd = float(mjd)
    if not time: time = time
    mjd += (time/24.)
    head = ""
    head = ("SRC: %s \n" % source +
                "RA: " + RA + "\n" + "DEC: " + DEC + "\n" +
                "MJD: %.6f \n" % (float(mjd)) +  
                "RF: 230.0000 GHz \n" +
                "FOVX: %i pix %f as \n" % (xdim, pdimas * xdim) +
                "FOVY: %i pix %f as \n" % (ydim, pdimas * ydim) +
                "------------------------------------\n" + hf)

    # Save
    np.savetxt(fname, outdata, header=head, fmt=fmts)
    return

In [33]:
npy_save_txt(pol_image, './text.txt')